<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/langchain_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain --progress-bar off
!pip install -qU langchainhub --progress-bar off
!pip install -qU duckduckgo-search --progress-bar off
!pip install -qU fireworks-ai --progress-bar off
!pip install -qU openai --progress-bar off

In [2]:
# @title load fireworks API key
#connect to google drive
from google.colab import drive
import json
import os


drive.mount('/content/drive')

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

os.environ["FIREWORKS_API_KEY"] = env['fireworks.ai']['apiKey']


Mounted at /content/drive


In [3]:
import openai
# This is required to make it work for old version of openai < 1
openai.api_base = "https://api.fireworks.ai/inference/v1"
openai.api_key = env['fireworks.ai']['apiKey']

In [4]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
# Turn this on only if you want to debug other wise it's hard to see the conversations.
set_debug(True)

In [55]:
from pydantic import BaseModel, Field
from langchain.agents import tool
from langchain.tools import BaseTool
from typing_extensions import Annotated
from typing import Literal, Optional, Type

TemperatureUnitSymbol = Literal["celcius", "fahrenheit"]

class GetCurrentWeatherInput(BaseModel):
    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: Optional[TemperatureUnitSymbol] = Field(description="The temperature unit value")

class GetCurrentWeather(BaseTool):
    name = "get_current_weather"
    description = "Get the current weather in a given location"
    args_schema: Type[BaseModel] = GetCurrentWeatherInput
    return_direct = True

    def _run(
        self, **payload
    ) -> str:
      location = payload.get("location")
      unit = payload.get("unit")
      weather_info = {
          "location": location,
          "temperature": "72",
          "unit": unit,
          "forecast": ["sunny", "windy"],
      }

      return f"Current location in {location} is 72 {unit}"



# Function calling

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function


functions = [get_current_weather]
tools = [{ "type": "function", "function": format_tool_to_openai_function(f)} for f in functions]

# Initialize a Fireworks chat model
# For function calling we cannot use ChatFireworks integration as it doesn't properly pass functions
llm = ChatOpenAI(model="accounts/fireworks/models/fw-function-call-34b-v0",
                 openai_api_key=env['fireworks.ai']['apiKey'],
                 openai_api_base="https://api.fireworks.ai/inference/v1",
                 # verbose=True,
                 temperature= 0, max_tokens= 1024,
                 model_kwargs={ "tools":tools }
                )


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.schema.output_parser import BaseLLMOutputParser
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnableLambda
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    BaseMessageChunk,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)

def format_agent_scratchpad_from_intermediate_steps(x):
  return format_to_openai_tool_messages(x["intermediate_steps"])

# having error when calling convert_message_to_dict from langchain (https://github.com/langchain-ai/langchain/blob/4c47f39fcb539fdeff6dd6d9b1f483cd9a1af69b/libs/community/langchain_community/adapters/openai.py#L104C5-L104C28)
# because when we use tool we are also submitting that to openai (with 'name' in payload which fireworks is not supported yet).
# By default langchain also send functions/tools as role to openai.
# fireworks doesn't support that so we are going to adjust the prompt
def prepare_prompt_for_llm(x):
  messages = []
  print('prepare_prompt_for_llm')
  for message in x.messages:
    if isinstance(message, ToolMessage):
      # remove the name as we don't want to pass that in fireworks.
      message = ToolMessage(content=message.content, tool_call_id=message.tool_call_id)

    messages.append(message)


  print(messages)
  return messages


agent = {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_agent_scratchpad_from_intermediate_steps(x),
    } | prompt | RunnableLambda(prepare_prompt_for_llm) | llm | OpenAIToolsAgentOutputParser()


agent_executor = AgentExecutor(agent=agent, tools=[get_current_weather], verbose=True)


agent_executor.invoke({"input": "what is the weather is sf?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what is the weather is sf?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "what is the weather is sf?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,agent_scratchpad>] Entering Chain run with input:
{
  "input": "what is the weather is sf?",
  "intermediate_steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,agent_scratchpad> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "what is the weather is sf?",
  "intermediate_steps": []
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableParallel<input,agent_scratchpad> > 4:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "what is the weather is sf?"
}
[chain/start] [1:chain:AgentExecutor >

{'input': 'what is the weather is sf?',
 'output': 'The current weather in sf is 72 fahrenheit. '}

# Regular LLM

In [ ]:
from langchain.llms import Fireworks
llm = Fireworks(
          fireworks_api_key=env['fireworks.ai']['apiKey'],
          model="accounts/fireworks/models/mixtral-8x7b-instruct",
          model_kwargs={"temperature": 0, "max_tokens": 4096},
      )

In [56]:
from langchain.prompts import PromptTemplate

REACT_PROMPT = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate(
          input_variables=["tools", "tool_name", "input", "agent_scratchpad"],
          template=REACT_PROMPT,
      )

In [57]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools.render import render_text_description_and_args

tools = [GetCurrentWeather()]

# We don't use `create_tool_calling_agent` because Mistral doesn't support tool calling (unless we use fire function)
# Instead we use regular `create_react_agent`
react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt, output_parser=None, tools_renderer=render_text_description_and_args)

# executes the logical steps we created
agent_executor = AgentExecutor(
  agent=react_agent,
  tools=tools,
  verbose=True,
  handle_parsing_errors=True,
  max_iterations = 5 # useful when agent is stuck in a loop
)

In [58]:
agent_executor.invoke({"input": "what is the weather is sf?"})

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what is the weather is sf?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [1:chain:AgentExecutor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] s] Exiting 

ValidationError: 1 validation error for GetCurrentWeatherInput
unit
  Field required [type=missing, input_value={'location': "{'location'... 'unit': 'fahrenheit'}"}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing